In [1]:
import numpy as np
import cv2
import keras
from keras.models import Model, Sequential
from keras.layers import Input, Convolution2D, ZeroPadding2D, MaxPooling2D, Flatten, Dense, Dropout, Activation
from PIL import Image
from keras.preprocessing.image import load_img, img_to_array
from keras.applications.imagenet_utils import preprocess_input
from keras.preprocessing import image
import matplotlib.pyplot as plt
from os import listdir


Using TensorFlow backend.


In [2]:
color = (67,67,67)

In [3]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [4]:
def preprocess_image(path):
    img = load_img(path, target_size=(224,224))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    return img

In [5]:
def loadVggFaceModel():
    model = Sequential()
    model.add(ZeroPadding2D((1,1),input_shape=(224,224, 3)))
    model.add(Convolution2D(64, (3,3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))	

    model.add(Convolution2D(4096, (7, 7), activation='relu'))
    model.add(Dropout(0.5))
    model.add(Convolution2D(4096, (1, 1), activation='relu'))
    model.add(Dropout(0.5))
    model.add(Convolution2D(2622, (1, 1)))
    model.add(Flatten())
    model.add(Activation('softmax'))
    
    from keras.models import model_from_json
    model.load_weights('vgg_face_weights.h5')
    
    vgg_face_descriptor = Model(inputs=model.layers[0].input, outputs=model.layers[-2].output)

    return vgg_face_descriptor


In [6]:
model = loadVggFaceModel()

In [7]:
peoples = '/home/sumanth/startup_weekend/cutom_images'
import time
mn= []

In [8]:
people = dict()

for file in listdir(peoples):
    print file
    name, extension = file.split('.')
    people[name] = model.predict(preprocess_image('/home/sumanth/startup_weekend/cutom_images/%s.jpg' % (name)))[0,:]
    print people[name]

Sumanth.jpg
[ 0.6240236   0.81834936  1.6128556  ... -0.42146948  2.0248094
  2.1833134 ]


In [9]:
def findCosineSimilarity(source, test):
    a = np.matmul(np.transpose(source), test)
    b = np.sum(np.multiply(source, source))
    c = np.sum(np.multiply(test, test))
    return 1 - (a / (np.sqrt(b) * np.sqrt(c)))

In [10]:
def speakWith(my_name):
    print my_name
    

In [11]:
cap = cv2.VideoCapture(0)
cap.set(3, 20000)
cap.set(4, 150000)
while(True):
    ret, img = cap.read()
#     cv2.imshow('img', img)
    faces = face_cascade.detectMultiScale(img, 1.3,5)
    for (x, y, w, h) in faces:
        x1 = x
        y1 = y
        x2 = x+w
        y2 = y+h
        cv2.rectangle(img, (x1,y1), (x2,y2), (255,255,255), 2)  
        if w > 130:
            detected_face = img[int(y):int(y+h), int(x):int(x+w)] #crop detected face
            detected_face = cv2.resize(detected_face, (224, 224)) #resize to 224x224
            
            img_pixels = image.img_to_array(detected_face)
            img_pixels = np.expand_dims(img_pixels, axis = 0)
            img_pixels /= 255
            captured_representation = model.predict(img_pixels)[0,:]
            
            found = 0
            
            
            for i in people:
                my_name = i
                
                representation = people[i]

                similarity = findCosineSimilarity(representation, captured_representation)
                #similarity =  cosine_similarity(representation, captured_representation)
                
                if(similarity < 0.40):
                    cv2.putText(img, my_name, (int(x+w+15), int(y-12)), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2) 
                    mn.append(my_name)
                    if len(mn) > 2 and (mn[-1] == mn[-2]):
                        speakWith(my_name)
                        mn=[]
                        
                    found = 1
                    break


            cv2.line(img,(int((x+x+w)/2),y+15),(x+w,y-20),color,1)
            cv2.line(img,(x+w,y-20),(x+w+10,y-20),color,1)

            if(found == 0): 
                cv2.putText(img, 'unknown', (int(x+w+15), int(y-12)), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
    cv2.imshow('img', img)
    if cv2.waitKey(1) & 0xFF == ord('q'): #press q to quit
        break
cap.release()
cv2.destroyAllWindows()

Sumanth
Sumanth
Sumanth
Sumanth
Sumanth
Sumanth
Sumanth
Sumanth
Sumanth
Sumanth
Sumanth
Sumanth
Sumanth
Sumanth
Sumanth
Sumanth
Sumanth


In [12]:
cap.release()
cv2.destroyAllWindows()